In [ ]:
!pip install -U transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
import transformers
import torch

model_id = "NousResearch/Meta-Llama-3-8B-Instruct"

# Llama 3 모델 로드

Llama-3 모델을 불러와보겠습니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_id
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             return_dict=True,
                                             torch_dtype='auto',
                                             device_map='cuda',
                                             do_sample=True,
                                            )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 

In [ ]:
# Delete any models previously created
del model, #accelerator

# Empty VRAM cache
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

양자화된 모델을 불러와보겠습니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_id
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             return_dict=True,
                                             torch_dtype='auto',
                                             device_map='cuda',
                                             do_sample=True,
                                             load_in_4bit=True,
                                            )


# 챗 템플릿 활용

- 역할 구분: 대화에 여러 참가자가 있을 수 있으며, 각 참가자의 발언을 구분할 수 있어야 합니다. 챗 템플릿은 이를 명확히 하기 위해 역할(role)을 정의합니다.
- 구조 제공: 템플릿을 사용하면 대화의 시작과 끝, 각 발언의 구분 등을 명확히 할 수 있습니다.
- 모델 학습 지원: 일관된 형식으로 데이터를 제공함으로써 모델이 데이터를 더 잘 이해하고, 훈련과 추론 과정에서 일관성 있는 성능을 발휘할 수 있도록 돕습니다.


### 챗 템플릿의 구성 요소
- 역할(Role): 주로 system, user, assistant와 같은 역할을 정의합니다.
    - system: 시스템이나 설정 관련 메시지.
    - user: 사용자의 입력 메시지.
    - assistant: 모델이나 AI의 응답 메시지.
- 특수 토큰: 각 역할이나 메시지의 경계를 나타내기 위해 사용됩니다. 예를 들어, <|endoftext|>는 텍스트의 끝을 나타내는 특수 토큰입니다.
- 프롬프트 텍스트: 실제로 모델이 입력받는 텍스트로, 각 역할에 해당하는 메시지들이 포함됩니다.

### 각 언어모델 별 템플릿 예시


#### OpenAI
```
<|system|>You are a helpful assistant.<|endoftext|>
<|user|>What is the capital of France?<|endoftext|>
<|assistant|>The capital of France is Paris.<|endoftext|>
```

#### Hugging Face
```
chat = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

```


In [ ]:
text = """
역사 퀴즈
```
나폴레옹의 유럽 정복
나폴레옹 보나파르트는 18세기 후반에서 19세기 초반까지 유럽을 정복한 프랑스의 군사 지도자였다. 그는 여러 차례의 전투에서 승리하여 유럽 대륙을 지배하려 했으나, 워털루 전투에서 패배하면서 몰락했다.
```
```
로마 제국의 붕괴
서로마 제국은 5세기 말에 게르만족의 침입으로 붕괴하였다. 이로 인해 중세 유럽의 혼란기가 시작되었고, 여러 작은 왕국들이 형성되었다. 로마 제국의 붕괴는 유럽 역사에 큰 영향을 미쳤다.

Q1. 나폴레옹의 유럽 정복과 로마 제국의 붕괴 사이에는 어떤 공통점과 차이점이 있을까요?
"""


chat = [
    { "role": "system", "content": " You are an artificial intelligence assistant that answers in Korean." },
    { "role": "user", "content": f"{text}" },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
token_ids = tokenizer.encode(prompt,
                             add_special_tokens=False,
                             return_tensors="pt")

# attention_mask 생성
attention_mask = token_ids.ne(tokenizer.pad_token_id).long()

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        attention_mask=attention_mask.to(model.device),
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        max_new_tokens=512,
        eos_token_id=[
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ],
    )
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

print(output)

output_ids는 모델이 생성한 토큰의 ID 시퀀스를 담고 있는 텐서입니다.
tolist() 메소드는 이 텐서를 파이썬 리스트로 변환합니다. output_ids는 배치 형태로 결과를 반환하기 때문에, tolist()[0]을 사용하여 첫 번째 결과를 선택하고, 입력 텍스트를 제외한 다른 부분을 가져오기 위해 [token_ids.size(1):]로 인덱싱합니다.

# 대화형 챗봇을 위해 응답 저장하기

In [ ]:
import torch

# 초기 대화 맥락 설정
chat = [
    { "role": "system", "content": "You are an artificial intelligence assistant that answers in Korean." }
]

def generate_response(model, tokenizer, chat):
    # 대화 맥락을 이용해 프롬프트 생성
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    tokenizer.pad_token_id = tokenizer.eos_token_id
    token_ids = tokenizer.encode(prompt,
                                 add_special_tokens=False,
                                 return_tensors="pt")

    # attention_mask 생성
    attention_mask = token_ids.ne(tokenizer.pad_token_id).long()

    # 모델 응답 생성
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            attention_mask=attention_mask.to(model.device),
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            max_new_tokens=512,
            eos_token_id=[
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")]
        )
    output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
    return output


In [ ]:
while True:
    user_input = input("User: ")
    chat.append({"role": "user", "content": user_input})

    # 모델에게 응답 요청
    response = generate_response(model, tokenizer, chat)

    # 모델 응답 출력
    print(f"AI: {response}")

    # 대화 맥락에 모델 응답 추가
    chat.append({"role": "assistant", "content": response})

    # 원하는 만큼 대화를 유지하기 위해 context length를 확인하고 필요시 제거
    if len(chat) > 20:  # 예: 대화 길이가 20을 초과하면 오래된 항목 제거
        chat = chat[-20:]

In [ ]:
chat